In [0]:
import keras
import numpy as np
import pandas as pd
import cv2
from keras import backend as K
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D, LocallyConnected1D
from keras.utils import np_utils
#from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, matthews_corrcoef
import itertools

Using TensorFlow backend.


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/ruhul-amin95/E.-coli/master/sigma_28(pwr).csv')
df.shape

(228, 80)

In [0]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [0]:
X = to_categorical(X)
y = to_categorical(y)

In [0]:
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.15)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, test_size=0.15)

In [0]:
print('test: ', np.sum(y_test, axis=0))
print('val: ', np.sum(y_val, axis=0))

test:  [13. 22.]
val:  [15. 14.]


In [0]:
# Model Architecture
model = Sequential()

model.add(Convolution1D(filters = 128, kernel_size= 3, activation='relu', input_shape=(79, 64)))
#model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.50))
model.add(Convolution1D(filters = 60, kernel_size = 2, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(Convolution1D(filters = 40, kernel_size = 2, activation='relu'))
model.add(Dropout(0.50))

model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
model.compile('rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=2, batch_size=24, verbose=1)

Train on 164 samples, validate on 29 samples
Epoch 1/2
164/164 [==============================] - 1s 6ms/step - loss: 0.3001 - acc: 0.8902 - val_loss: 0.1604 - val_acc: 0.9655
Epoch 2/2
164/164 [==============================] - 0s 272us/step - loss: 0.1179 - acc: 0.9695 - val_loss: 0.1225 - val_acc: 0.9655


In [0]:
y_true = np.argmax(y_test, axis=1)
y_pred = model.predict_classes(X_test)

In [0]:
print(accuracy_score(y_true, y_pred) * 100)
print(classification_report(y_true, y_pred))
print(matthews_corrcoef(y_true, y_pred))

97.14285714285714
              precision    recall  f1-score   support

           0       0.93      1.00      0.96        13
           1       1.00      0.95      0.98        22

   micro avg       0.97      0.97      0.97        35
   macro avg       0.96      0.98      0.97        35
weighted avg       0.97      0.97      0.97        35

0.9414688716912718


In [0]:
model.save('drive/My Drive/sigma_28(97.14).h5')